In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
from yomi.historical_record import fetch_historical_elo, compute_elo

In [11]:
matches = fetch_historical_elo()
matches.head()

,set_number,event,match_date,elo_1_before,set_win_1,player_1,player_2,set_win_2,elo_2_before
0,1,Summer Smash,2014-07-06,1500.0,1,ntillerman,Felix,0,1500.0
1,2,Summer Smash,2014-07-06,1500.0,1,Mallorean_Thug,Nathan,0,1500.0
2,3,Summer Smash,2014-07-06,1500.0,1,Bob199,Djister,0,1500.0
3,4,Summer Smash,2014-07-06,1500.0,1,Xom,rybalko,0,1500.0
4,5,Summer Smash,2014-07-06,1500.0,1,Redless,LK4O4,0,1500.0


In [22]:
results = []
best_result = None

for low_k in range(35, 45, 1):
    for high_k in range(90, 105, 1):
        for k_cutoff in range(40, 70, 1):
            elo = compute_elo(matches, low_k=low_k, high_k=high_k, k_cutoff=k_cutoff)

            elo["points_diff"] = elo.elo_before_1 - elo.elo_before_2
            elo["win_chance"] = 1 / (1 + (-elo.points_diff / 1135.77).rpow(10))
            squared_error = (
                (elo.win_chance - elo.set_win_1).pow(2)
                + ((1 - elo.win_chance) - (elo.set_win_2)).pow(2)
            ).sum()
    
            result = (squared_error, low_k, high_k, k_cutoff)
            if best_result is None or result < best_result:
                best_result = result
                print("Found new best result", result)
            results.append(result)
            if low_k == high_k:
                break

Found new best result (2379.8403683279716, 35, 90, 40)
Found new best result (2378.9200021110937, 35, 90, 46)
Found new best result (2377.8831153432943, 35, 90, 47)
Found new best result (2377.8177110926854, 35, 90, 48)
Found new best result (2377.5234092576175, 35, 90, 50)
Found new best result (2377.347742158676, 35, 90, 51)
Found new best result (2377.0963021398316, 35, 90, 54)
Found new best result (2375.4324723858726, 35, 90, 55)
Found new best result (2375.372727880498, 35, 91, 55)
Found new best result (2375.3227792813477, 35, 92, 55)
Found new best result (2375.2824531781725, 35, 93, 55)
Found new best result (2375.2515806396473, 35, 94, 55)
Found new best result (2375.229997054942, 35, 95, 55)
Found new best result (2375.21754198238, 35, 96, 55)
Found new best result (2375.214059004836, 35, 97, 55)
Found new best result (2375.191926629377, 36, 94, 55)
Found new best result (2375.1706130680523, 36, 95, 55)
Found new best result (2375.1583952819137, 36, 96, 55)
Found new best re

In [23]:
min(results)

(2375.0511659058793, 40, 97, 55)